In [1]:
import tensorflow as tf
import numpy as np
%matplotlib qt
import matplotlib.pyplot as plt

def add_layer(inputs, in_size , out_size , activation_function = None):
    # in_size行，out_size列
    Weights = tf.Variable(tf.random_normal([in_size , out_size]))
    # 1行 ， out_size列
    biases = tf.Variable(tf.zeros([1 , out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs , Weights) + biases
    if activation_function is None :
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return (outputs)

## 构建数据，线性排列，从-1到1，300个单位；[:, np.newaxis]用来加一个维度，有300行

In [2]:

x_data = np.linspace(-1 , 1 ,300)[:, np.newaxis]
#加入噪点,正态分布，均值是0，标准差是0.05，格式是x_data.shape
noise = np.random.normal(0 , 0.05 , x_data.shape)
#y_data是x_data的二次方
y_data = np.square(x_data) - 0.5 + noise

## 用于把值输入train_step , None表示无论给多少给例子都可以

In [3]:
xs = tf.placeholder(tf.float32,[None , 1])
ys = tf.placeholder(tf.float32,[None , 1])

#构建layer,输入层只有1个神经元x，输出层也只有1个y，假设隐藏层有10个神经元

In [4]:
l1 = add_layer(xs , 1 , 10 , activation_function = tf.nn.relu)
prediction = add_layer(l1 , 10 , 1 , activation_function = None)
#reduction_indices = [1]表示行求和，前面数据有300行
loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction) , reduction_indices = [1]))

## 0.1指的是学习效率，通常小于1

In [5]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

## 初始化

In [6]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

## 显示真实数据

In [7]:
fig = plt.figure()
ax = fig.add_subplot( 1 , 1 , 1 )
ax.scatter( x_data , y_data )
plt.ion()
plt.show()

## 进行训练，输出loss。改写后显示拟合的曲线

In [8]:
for i in range(1000):
    sess.run(train_step , feed_dict = {xs : x_data , ys : y_data})
    if i % 50==0:
        #print(sess.run(loss , feed_dict = {xs : x_data , ys : y_data}))
        try :
            ax.lines.remove( lines[0])
        except Exception:
            pass
        prediction_value = sess.run( prediction , feed_dict = { xs : x_data })
        lines = ax.plot( x_data , prediction_value , 'r-' , lw = 5 )
        plt.pause(0.1)